In [87]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA


In [88]:
# Fonction pour télécharger des données historiques
def download_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Date'] = data.index
    return data

# Exemple de téléchargement de données
ticker = 'AAPL'  # Symbole boursier de l'action Apple
start_date = '2020-01-01'
end_date = '2023-01-01'

data = download_data(ticker, start_date, end_date)
print(data.head())


[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2020-01-02  74.059998  75.150002  73.797501  75.087502  73.059418  135480400   
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.349121  146322800   
2020-01-06  73.447502  74.989998  73.187500  74.949997  72.925629  118387200   
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.582657  108872000   
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.750244  132079200   

                 Date  
Date                   
2020-01-02 2020-01-02  
2020-01-03 2020-01-03  
2020-01-06 2020-01-06  
2020-01-07 2020-01-07  
2020-01-08 2020-01-08  


In [89]:
#definitions des indicateurs
def RSI(data, period):
    series = pd.Series(data)
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


def MACD(data, short_period=12, long_period=26, signal_period=9):
    short_ema = pd.Series(data).ewm(span=short_period, adjust=False).mean()
    long_ema = pd.Series(data).ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def EMA(data, period):
    return data.ewm(span=period, adjust=False).mean()

def SMA(data, period):
    return data.rolling(window=period).mean()


In [90]:
params1 = 20
params2 = 50
params3 = 30

In [91]:
class Strat(Strategy):
    parametre1 = params1  # exemple de valeur par défaut
    parametre2 = params2  # exemple de valeur par défaut

    def init(self):
        # Initialisation des moyennes mobiles exponentielles
        self.ema1 = self.I(EMA, pd.Series(self.data.Close), self.parametre1)
        self.ema2 = self.I(EMA, pd.Series(self.data.Close), self.parametre2)

    def next(self):
        # Croisement des EMA pour les signaux d'achat et de vente
        if crossover(self.ema1, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.sell()



In [92]:
def perform_backtest(data):
    Strat.parametre1 = params1
    Strat.parametre2 = params2
    print(Strat.parametre1, Strat.parametre2)
    bt = Backtest(data, Strat, cash=10000, commission=.002)
    output = bt.run()
    bt.plot()
    return output

# Exécution du backtest
result = perform_backtest(data)
print(result)


c:\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value

20 50
Start                     2020-01-02 00:00:00
End                       2022-12-30 00:00:00
Duration                   1093 days 00:00:00
Exposure Time [%]                   99.206349
Equity Final [$]                 16477.401873
Equity Peak [$]                  23338.200701
Return [%]                          64.774019
Buy & Hold Return [%]               73.038109
Return (Ann.) [%]                   18.112604
Volatility (Ann.) [%]               44.926232
Sharpe Ratio                         0.403163
Sortino Ratio                        0.722745
Calmar Ratio                         0.578587
Max. Drawdown [%]                  -31.304917
Avg. Drawdown [%]                   -5.533646
Max. Drawdown Duration      361 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      65.040172
Worst Trade [%]                     65.040172
Avg. Trade [%]              